In [5]:
import pandas as pd
import os

In [6]:
mainindir="/scratch/Shares/dowell/dbnascent/out/meta_analysis/snpconnecter/outputdata/"
rootname="EFO_0000565"




# Read in all the data that came from the GWAStogenes.sh script 

In [7]:
def snpclosestgenedf(mainindir, rootname, tissue):
    snptobedfile=mainindir+rootname+"_"+tissue+"/"+rootname+".snp_to_bed.txt"
    snptobeddf = pd.read_csv(snptobedfile, sep="\t")
    closestgenetosnp=mainindir+rootname+"_"+tissue+"/"+rootname+".closestgene.sorted.bed"
    closestgenetosnpdf = pd.read_csv(closestgenetosnp, sep="\t", names=["snp_chr", "snp_start", "snp_stop", "closest_gene_chr", "closest_gene_start", "closest_gene_stop", "closest_gene_name","score", "strand", "closest_distance"], index_col=False)
    closestgenetosnpdf[["closest_gene", "closest_transcript_id"]] = closestgenetosnpdf['closest_gene_name'].str.split(':', n=1, expand=True)
    masterdf = snptobeddf.merge(closestgenetosnpdf, on=["snp_chr", "snp_start"])
    print ("unique snps in bidirectionals", masterdf["rsID"].nunique())
    return masterdf
    

def merge_dfs_outer(mainindir, rootname, tissue):
    snptobedfile=mainindir+rootname+"_"+tissue+"/"+rootname+".snp_to_bed.txt"
    bedtobidirfile=mainindir+rootname+"_"+tissue+"/"+rootname+".snpinbidir.df"
    bidirtogenefile=mainindir+rootname+"_"+tissue+"/"+rootname+".genelist.txt.df"
    snptobeddf = pd.read_csv(snptobedfile, sep="\t")
    checkedbidirs = readsnpsandgeneschecked()
    bedtobidirdf = pd.read_csv(bedtobidirfile)
    bidirtogenedf = pd.read_csv(bidirtogenefile)
    bedtobidirdf["bidirtesed_for_links"]= bedtobidirdf["bidirname"].isin(checkedbidirs)
    masterdf = snptobeddf.merge(bedtobidirdf, on=["snp_chr", "snp_start"], how="outer")
    masterdf = masterdf.merge(bidirtogenedf, left_on="bidirname", right_on="transcript_2", how="outer")
    masterdf_mini = masterdf[["riskAllele", "rsID", 'snp_chr', 'snp_start', "bidirname", "distance_tss", "transcript_1", "gene", "bidirtesed_for_links"]].drop_duplicates()
    closestgenetosnpdf = snpclosestgenedf(mainindir, rootname, tissue)
    backtoallsnps = closestgenetosnpdf.merge(masterdf_mini, how="outer", on=["snp_chr", "snp_start", "riskAllele", "rsID"])
    backtoallsnps["snp_in_gene"] = backtoallsnps["closest_distance"]==0
    return backtoallsnps

#
def readsnpsandgeneschecked():
    #only genes and snps with expression were checked for correlation
    filename="/scratch/Shares/dowell/rutendo/projects/DBNascent_Analysis/data/counts/normalized/genes_inter_bidir_filtered.tsv.gz"
    checkedforlinksdf = pd.read_csv(filename, compression='gzip', sep="\t",low_memory=False)
    bidircheckedforlinksdf = checkedforlinksdf[checkedforlinksdf["gene_transcript"].str.startswith("chr")]
    return bidircheckedforlinksdf["gene_transcript"].unique()
    
    

In [8]:
tissue="blood"
allsnps_one_tissue = merge_dfs_outer(mainindir, rootname, tissue)
    

unique snps in bidirectionals 7805


In [9]:
allsnps_one_tissue

,Unnamed: 0,riskAllele,locations,snp_chr,snp_start,rsID,change,stop,snp_stop,closest_gene_chr,...,strand,closest_distance,closest_gene,closest_transcript_id,bidirname,distance_tss,transcript_1,gene,bidirtesed_for_links,snp_in_gene
0,0,rs7090445-?,10:61961417,chr10,61961417,rs7090445,?,61961418,61961418,chr10,...,+,88689,ARID5B,NM_001244638.2,chr10:61961221-61961505,NaN,NaN,NaN,False,False
1,63,rs7090445-?,10:61961417,chr10,61961417,rs7090445,?,61961418,61961418,chr10,...,+,88689,ARID5B,NM_001244638.2,chr10:61961221-61961505,NaN,NaN,NaN,False,False
2,1,rs2665658-A,2:60599667,chr2,60599667,rs2665658,A,60599668,60599668,chr2,...,-,71883,BCL11A,NR_175964.1,NaN,NaN,NaN,NaN,NaN,False
3,2,rs2665658-A,2:60599667,chr2,60599667,rs2665658,A,60599668,60599668,chr2,...,-,71883,BCL11A,NR_175964.1,NaN,NaN,NaN,NaN,NaN,False
4,3,rs4869742-T,6:151586613,chr6,151586613,rs4869742,T,151586614,151586614,chr6,...,+,0,CCDC170,NM_025059.4,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13082,2110,rs131821-AT,22:50511648,chr22,50511648,rs131821,AT,50511649,50511649,chr22,...,+,0,NCAPH2,NM_014551.5,chr22:50511408-50511748,NaN,NaN,NaN,False,True
13083,2111,rs11715604-?,3:136870707,chr3,136870707,rs11715604,?,136870708,136870708,chr3,...,-,9395,NCK1-DT,NR_110175.1,chr3:136869554-136871154,NaN,NaN,NaN,False,False
13084,2112,rs9421684-A,10:80702590,chr10,80702590,rs9421684,A,80702591,80702591,chr10,...,+,48857,LINC02655,NR_155749.1,NaN,NaN,NaN,NaN,NaN,False
13085,2113,rs11187157-C,10:92742487,chr10,92742487,rs11187157,C,92742488,92742488,chr10,...,+,46841,HHEX,NM_002729.5,NaN,NaN,NaN,NaN,NaN,False


In [10]:
ingenesnpsdf = allsnps_one_tissue[allsnps_one_tissue["closest_distance"]==0]
notgenesnpsdf = allsnps_one_tissue[allsnps_one_tissue["closest_distance"]!=0]
print ("unique snps in genes", ingenesnpsdf["rsID"].nunique())
print ("unique snps not in genes", notgenesnpsdf["rsID"].nunique())
ingenesnpsdf

unique snps in genes 3245
unique snps not in genes 4560


,Unnamed: 0,riskAllele,locations,snp_chr,snp_start,rsID,change,stop,snp_stop,closest_gene_chr,...,strand,closest_distance,closest_gene,closest_transcript_id,bidirname,distance_tss,transcript_1,gene,bidirtesed_for_links,snp_in_gene
4,3,rs4869742-T,6:151586613,chr6,151586613,rs4869742,T,151586614,151586614,chr6,...,+,0,CCDC170,NM_025059.4,NaN,NaN,NaN,NaN,NaN,True
7,6,rs72832868-A,2:110850099,chr2,110850099,rs72832868,A,110850100,110850100,chr2,...,+,0,ACOXL,NM_001365487.2,NaN,NaN,NaN,NaN,NaN,True
8,7,rs114959229-T,22:46469808,chr22,46469808,rs114959229,T,46469809,46469809,chr22,...,-,0,CELSR1,NM_014246.4,NaN,NaN,NaN,NaN,NaN,True
9,8,rs139440276-T,3:66312665,chr3,66312665,rs139440276,T,66312666,66312666,chr3,...,+,0,SLC25A26,NM_001379210.1,chr3:66312484-66313196,NaN,NaN,NaN,False,True
11,10,rs59382229-G,1:241338685,chr1,241338685,rs59382229,G,241338686,241338686,chr1,...,-,0,RGS7,NM_001350116.1,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13079,2107,rs181181503-C,12:74276187,chr12,74276187,rs181181503,C,74276188,74276188,chr12,...,-,0,LINC02882,NR_038300.1,NaN,NaN,NaN,NaN,NaN,True
13080,2108,rs1576377-C,13:60686390,chr13,60686390,rs1576377,C,60686391,60686391,chr13,...,+,0,LINC00378,NR_047003.1,NaN,NaN,NaN,NaN,NaN,True
13081,2109,rs73005220-G,19:16161878,chr19,16161878,rs73005220,G,16161879,16161879,chr19,...,-,0,CIB3,NM_054113.4,chr19:16161672-16162168,NaN,NaN,NaN,False,True
13082,2110,rs131821-AT,22:50511648,chr22,50511648,rs131821,AT,50511649,50511649,chr22,...,+,0,NCAPH2,NM_014551.5,chr22:50511408-50511748,NaN,NaN,NaN,False,True


In [11]:
allsnps_one_tissue.columns

Index(['Unnamed: 0', 'riskAllele', 'locations', 'snp_chr', 'snp_start', 'rsID',
       'change', 'stop', 'snp_stop', 'closest_gene_chr', 'closest_gene_start',
       'closest_gene_stop', 'closest_gene_name', 'score', 'strand',
       'closest_distance', 'closest_gene', 'closest_transcript_id',
       'bidirname', 'distance_tss', 'transcript_1', 'gene',
       'bidirtesed_for_links', 'snp_in_gene'],
      dtype='object')

# Output what we know about all snps in this GWAS catalog

In [22]:

def informationforpub(tissue, backtoallsnps):
    print ("riskAlleles", len(backtoallsnps["riskAllele"].unique()))
    print ("snp locations", len(backtoallsnps["locations"].unique()))
    print ("rsID", len(backtoallsnps["rsID"].unique()))
    print ("closest genes", len(backtoallsnps["closest_gene"].unique()))
    ingenesnpsdf = backtoallsnps[backtoallsnps["closest_distance"]==0]
    print ("snps in genes", ingenesnpsdf["rsID"].nunique())
    print ("closest genes when snp is in a gene", len(ingenesnpsdf["closest_gene"].unique()))
    notingenesnpsdf = backtoallsnps[backtoallsnps["closest_distance"]!=0]
    print ("snps not in  genes", notingenesnpsdf["rsID"].nunique())
    print ("closest genes when snp is not in a gene", len(notingenesnpsdf["closest_gene"].unique()))
    
    inabidir = backtoallsnps[~backtoallsnps["bidirname"].isna()]
    print ("snps in any birectional", inabidir["rsID"].nunique())
    inabidirnotgene = notingenesnpsdf[~notingenesnpsdf["bidirname"].isna()]
    inabidirinagene = ingenesnpsdf[~ingenesnpsdf["bidirname"].isna()]
    inabidirinagenetestedforsnps = backtoallsnps[backtoallsnps["bidirtesed_for_links"]==True]
    inabidirinagenetestedforsnps2 = inabidirnotgene[inabidirnotgene["bidirtesed_for_links"]==True]
    inabidirinagenetestedforsnps3 = inabidirinagene[inabidirinagene["bidirtesed_for_links"]==True]
    print ("snps in any birectional that's not in a gene", inabidirnotgene["rsID"].nunique())
    print ("snps in any birectional that's in a gene", inabidirinagene["rsID"].nunique())
    #/scratch/Shares/dowell/rutendo/projects/DBNascent_Analysis/data/counts/normalized/genes_inter_bidir_filtered.tsv.gz
    print ("snps in any birectional that was tested for correlations", inabidirinagenetestedforsnps["rsID"].nunique())
    print ("snps in any birectional that was tested for correlations (snp not in gene)", inabidirinagenetestedforsnps2["rsID"].nunique())
    print ("snps in any birectional that was tested for correlations (snp in gene)", inabidirinagenetestedforsnps3["rsID"].nunique())  
    print ("closeset genes when snp in any birectional", len(inabidirnotgene["closest_gene"].unique()))
    inabidirlinkedtoagene = inabidirnotgene[~inabidirnotgene["transcript_1"].isna()]
    print ("snps in any birectional and snp linked to gene", inabidirlinkedtoagene["rsID"].nunique())
    print ("closeset genes when snp in any birectional and snp linked to gene", len(inabidirlinkedtoagene["closest_gene"].unique()))
    print ("nascentlinked genes when snp in any birectional and snp linked to gene", len(inabidirlinkedtoagene["transcript_1"].unique()))
    inabidirlinkedtoagene = inabidirnotgene[~inabidirnotgene["transcript_1"].isna()]
    repogene = [gene for gene in backtoallsnps["gene"].unique() if gene not in backtoallsnps["closest_gene"].unique()]
    closestgene = [gene for gene in backtoallsnps["closest_gene"].unique() if gene not in backtoallsnps["gene"].unique()]
    both = [gene for gene in backtoallsnps["closest_gene"].unique() if gene in backtoallsnps["gene"].unique()]
    print ("for all snps intergentic and geneic")
    print ("\tgenes that are nascentlinked and are not closeset genes", len(repogene))
    print ("\tgenes that are both in closeset genes and nascentlinked", len(both))
    print ("\tgenes that are closeset genes not nascentlinked", len(closestgene))
    repogene = [gene for gene in notingenesnpsdf["gene"].unique() if gene not in notingenesnpsdf["closest_gene"].unique()]
    closestgene = [gene for gene in notingenesnpsdf["closest_gene"].unique() if gene not in notingenesnpsdf["gene"].unique()]
    both = [gene for gene in notingenesnpsdf["closest_gene"].unique() if gene in notingenesnpsdf["gene"].unique()]
    print ("for snps that are intergentic")
    print ("\tgenes that are nascentlinked and are not closeset genes", len(repogene))
    print ("\tgenes that are both in closeset genes and nascentlinked", len(both))
    print ("\tgenes that are closeset genes not nascentlinked", len(closestgene))
    repogene = [gene for gene in inabidirnotgene["gene"].unique() if gene not in inabidirnotgene["closest_gene"].unique()]
    closestgene = [gene for gene in inabidirnotgene["closest_gene"].unique() if gene not in inabidirnotgene["gene"].unique()]
    both = [gene for gene in inabidirnotgene["closest_gene"].unique() if gene in inabidirnotgene["gene"].unique()]
    print ("for only snps in bidirs")
    print ("\tgenes that are nascentlinked and are not closeset genes", len(repogene))
    print ("\tgenes that are both in closeset genes and nascentlinked", len(both))
    print ("\tgenes that are closeset genes not nascentlinked", len(closestgene))

In [23]:
informationforpub(tissue, allsnps_one_tissue)

riskAlleles 8513
snp locations 7805
rsID 7805
closest genes 1386
snps in genes 3245
closest genes when snp is in a gene 676
snps not in  genes 4560
closest genes when snp is not in a gene 949
snps in any birectional 1061
snps in any birectional that's not in a gene 579
snps in any birectional that's in a gene 482
snps in any birectional that was tested for correlations 393
snps in any birectional that was tested for correlations (snp not in gene) 391
snps in any birectional that was tested for correlations (snp in gene) 2
closeset genes when snp in any birectional 349
snps in any birectional and snp linked to gene 126
closeset genes when snp in any birectional and snp linked to gene 99
nascentlinked genes when snp in any birectional and snp linked to gene 259
for all snps intergentic and geneic
	genes that are nascentlinked and are not closeset genes 217
	genes that are both in closeset genes and nascentlinked 43
	genes that are closeset genes not nascentlinked 1343
for snps that are i

# output about genes that have a snp in them and have a snp in a bidireciontl linked to them

In [24]:
inabidirnotgene = allsnps_one_tissue[~allsnps_one_tissue["bidirname"].isna()]
inabidirnotgene = inabidirnotgene[~inabidirnotgene["transcript_1"].isna()]
inabidirnotgene = inabidirnotgene[inabidirnotgene["closest_distance"]!=0]

In [25]:
repogene = [gene for gene in inabidirnotgene["gene"].unique() if gene not in inabidirnotgene["closest_gene"].unique()]
closestgene = [gene for gene in allsnps_one_tissue["closest_gene"].unique() if gene not in allsnps_one_tissue["gene"].unique()]
both = [gene for gene in allsnps_one_tissue["closest_gene"].unique() if gene in allsnps_one_tissue["gene"].unique()]




In [26]:
len(repogene)

236

In [27]:
len(both)

43

In [28]:
len(closestgene)

1343

# A intergenic snp we say regulates a gene and that gene has already got another snp inside it that is said to affect this disorder

In [29]:
repogene_ingenetoo = [gene for gene in inabidirnotgene["gene"].unique() if gene in allsnps_one_tissue["closest_gene"].unique()]
repogene_ingenetoo


['LINC00977',
 'IKZF1',
 'MDS2',
 'MIR8059',
 'FCRL3',
 'GOLGA2P11',
 'RGS2-AS1',
 'UCHL5',
 'ASB2',
 'CDKN2A-DT',
 'CDKN2A',
 'CDKN2B-AS1',
 'IFNA5',
 'RCL1',
 'PLXNA2',
 'LNC-LBCS',
 'SIM1',
 'AGR2',
 'LINC02227',
 'TLX3',
 'LINC01121',
 'LOC100129434',
 'RFPL1S',
 'LINC02211',
 'LINC01053',
 'LINC00578',
 'ABTB2',
 'ASRGL1',
 'HMX3',
 'WTAPP1',
 'IL15RA',
 'PRKCQ',
 'SFMBT2',
 'MAP1LC3B2',
 'MED13L',
 'TMEM132C',
 'AFF3',
 'MKRN7P',
 'LINC02313',
 'TMEM260',
 'SLC24A3',
 'BALR6',
 'LNX1-AS2']

In [30]:
len(repogene_ingenetoo)

43

In [31]:
closestgenecaugthitT=0
closestgenecaugthitF=0
closesttothissnpcaugthitT=0
closesttothissnpcaugthitF=0
for thisgene in repogene_ingenetoo:
    onegeneintergenicallinfo=notgenesnpsdf[notgenesnpsdf["gene"]==thisgene]
    onegenegenicallinfo=onegeneintergenicallinfo[onegeneintergenicallinfo["closest_gene"]==thisgene]
    closesttothissnpcaugthit = len([agene for agene in onegeneintergenicallinfo["gene"].unique() if agene in onegenegenicallinfo["closest_gene"].unique()])==1
    closestgenecaugthit = len([agene for agene in onegeneintergenicallinfo["gene"].unique() if agene in notgenesnpsdf["closest_gene"].unique()])==1
    if closestgenecaugthit:
        closestgenecaugthitT=closestgenecaugthitT+1
    else:
        closestgenecaugthitF=closestgenecaugthitF+1
    if closesttothissnpcaugthit:
        closesttothissnpcaugthitT=closesttothissnpcaugthitT+1
    else:
        closesttothissnpcaugthitF=closesttothissnpcaugthitF+1

    print (thisgene, "did closest to this snp catch this gene too?", closesttothissnpcaugthit)
    print (thisgene, "did closest to any snp catch this gene too?", closestgenecaugthit)
    #print("      snp intergenic closest_gene gene (snp to gene distance)", onegeneintergenicallinfo[["snp_start", "closest_gene", "closest_distance"]])#
    #print("      snp intergenic repo predicted gene (enhancer mu to gene distance)", onegeneintergenicallinfo[["snp_start", "gene", "distance_tss"]])#["snp_start", "gene", "distance_tss"]
    #print("      snp gene internal to gene", onegenegenicallinfo[["snp_start", "closest_gene", "closest_distance"]])#["snp_start", "closest_gene", "closest_distance"]
    
    
print ("    ")    
print ("closest to this snp catch this gene too")    
print ("True", closesttothissnpcaugthitT)
print ("False", closesttothissnpcaugthitF)
print ("closest to any snp catch this gene too")    
print ("True", closestgenecaugthitT)
print ("False", closestgenecaugthitF)

LINC00977 did closest to this snp catch this gene too? True
LINC00977 did closest to any snp catch this gene too? True
IKZF1 did closest to this snp catch this gene too? True
IKZF1 did closest to any snp catch this gene too? True
MDS2 did closest to this snp catch this gene too? True
MDS2 did closest to any snp catch this gene too? True
MIR8059 did closest to this snp catch this gene too? True
MIR8059 did closest to any snp catch this gene too? True
FCRL3 did closest to this snp catch this gene too? True
FCRL3 did closest to any snp catch this gene too? True
GOLGA2P11 did closest to this snp catch this gene too? True
GOLGA2P11 did closest to any snp catch this gene too? True
RGS2-AS1 did closest to this snp catch this gene too? True
RGS2-AS1 did closest to any snp catch this gene too? True
UCHL5 did closest to this snp catch this gene too? False
UCHL5 did closest to any snp catch this gene too? True
ASB2 did closest to this snp catch this gene too? True
ASB2 did closest to any snp catc

In [32]:
backtoallsnps.columns

NameError: name 'backtoallsnps' is not defined